In [ ]:
!pip install libtorrent --quiet

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import os
import gc
import re
import time
import shutil
import libtorrent as lt

In [ ]:
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
def validate_magnet_link(magnet_link, exact=False):
    pattern_str = (
        r"^magnet:\?xt=urn:[a-z0-9]+:[a-z0-9]{32,40}&dn=.+&tr=.+$"
        if exact
        else r"magnet:\?xt=urn:[a-z0-9]+:[a-z0-9]{32,40}&dn=.+&tr=.+"
    )
    pattern = re.compile(pattern_str, re.I | re.M)
    return bool(pattern.match(magnet_link))

In [ ]:
def load_magnet_links(file_name: str) -> set:
    try:
        with open(file_name) as f:
            lines = [line.strip() for line in f]

        magnet_links = [line for line in lines if validate_magnet_link(line)]
        print(f"Loaded {len(magnet_links)} magnet links from {file_name}")
        return set(magnet_links)

    except FileNotFoundError:
        print(f"File not found: {file_name}")
        return set([])

In [ ]:
def get_magnet_links(
    magnet_links: set,
) -> set:
    count = 0

    while True:
        try:
            link = input("Enter a magnet link or 'exit' to quit: ")

            if link in ["exit", "exıt", "quit", "q"]:
                break

            if validate_magnet_link(link, exact=True):
                if link not in magnet_links:
                    magnet_links.append(link)
                    print(f"Added magnet link: {link}")
                    count += 1

                else:
                    print(f"Magnet link already added: {link}")

            else:
                print(f"Invalid magnet link: {link}")

        except KeyboardInterrupt:
            break

    print(f"Added {count} magnet links")

    return magnet_links

In [ ]:
def update_txt_file(file_name: str, magnet_links: set):
    with open(file_name, "w") as f:
        f.write("\n".join(magnet_links))

    print(f"{time.strftime('%H:%M:%S')} - Txt file updated -> {file_name}")

In [ ]:
class Torrent:
    def __init__(
        self,
        ses: lt.session,
        magnet_link: str,
    ):
        self.magnet_link = magnet_link
        self.save_path = "/content/Torrents"
        self.timestamp = time.time()
        self.handle = self.add_torrent(ses)
        self.name = self.handle.status().name

    def add_torrent(self, ses: lt.session):
        atp = lt.parse_magnet_uri(self.magnet_link)
        atp.save_path = self.save_path
        atp.storage_mode = lt.storage_mode_t.storage_mode_sparse

        return ses.add_torrent(atp)

    def remove_torrent(self, ses: lt.session):
        ses.remove_torrent(self.handle)
        print(f"Torrent successfully removed: {self.name}")

    def remove_stale_torrents(self, ses: lt.session):
        if time.time() - self.timestamp > 7200:
            self.remove_torrent(ses)
            return True

        return False

    def get_status(self):
        return self.handle.status()

    def get_folder_name(self, lower=True):
        folder_name = (
            self.name[:50].split(".")[0]
            if "." in self.name
            else self.name[:50].split(" ")[0]
        )

        return folder_name.lower() if lower else folder_name

    def get_torrent_name(self):
        return self.handle.status().name

    def remove_the_link_from_the_list(self, magnet_links):
        magnet_links.remove(self.magnet_link)

    def remove_files(self):
        shutil.rmtree(os.path.join(self.save_path, self.get_folder_name()))
        print(f"Files successfully removed from session: {self.get_folder_name()}")

    def move_files(self, destination):
        shutil.move(
            os.path.join(self.save_path, self.get_folder_name()),
            os.path.join(destination, self.get_folder_name()),
        )

    def move_video_files(self, destination):
        torrent_destination = os.path.join(destination, self.get_folder_name())
        create_directory(torrent_destination)

        moved_files = 0  # Initialize a counter for moved files

        source_folder = os.path.join(self.save_path, self.name)

        for file in os.listdir(source_folder):
            if any(
                file.lower().endswith(ext) for ext in [".mp4", ".mkv", ".avi", ".mov"]
            ):
                source_path = os.path.join(source_folder, file)
                destination_path = os.path.join(torrent_destination, file)

                shutil.move(source_path, destination_path)
                moved_files += 1  # Increment the counter

        # Remove the source folder after moving files
        shutil.rmtree(source_folder)

        print(f"Moved {moved_files} video files to: {torrent_destination}")

In [ ]:
def create_torrents(ses: lt.session, magnet_links: set, downloads: list) -> set:
    for magnet_link in magnet_links:
        if magnet_link in [torrent.magnet_link for torrent in downloads]:
            print(f"Torrent already added to session: {magnet_link[:80]}")

        else:
            downloads.append(Torrent(ses, magnet_link))
            # increase this to stability
            time.sleep(5)

In [ ]:
def check_already_downladed(
    ses: lt.session,
    downloads: list,
    destination: str,
    magnet_links: set,
    magnet_links_txt: str,
):
    for torrent in downloads:
        folder_name = torrent.get_folder_name()

        if os.path.exists(os.path.join(destination, folder_name)):
            downloadeds = os.listdir(os.path.join(destination, folder_name))

            if any(torrent.name in downloaded for downloaded in downloadeds):
                torrent.remove_torrent(ses)
                torrent.remove_files()
                torrent.remove_the_link_from_the_list(magnet_links)
                update_txt_file(magnet_links_txt, magnet_links)
                torrent_name = torrent.name
                print(
                    f"Torrent already downloaded: {torrent_name} and removed from session!!!\nIt's already in {downloadeds}."
                )
                gc.collect()

        # if the folder not created create a folder
        else:
            create_directory(os.path.join(destination, folder_name))
            print(
                f"Directory created for file: {torrent.name[:80]} to {os.path.join(destination, folder_name)}"
            )

In [ ]:
def download_torrents(
    ses: lt.session,
    downloads: list,
    destination: str,
    magnet_links: set,
    magnet_links_txt: str,
):
    while downloads:
        for torrent in downloads.copy():
            # check if torrent is stale
            if torrent.remove_stale_torrents(ses):
                torrent.remove_files()
                torrent.remove_the_link_from_the_list(magnet_links)
                update_txt_file(magnet_links_txt, magnet_links)
                torrent_name = torrent.name
                downloads.remove(torrent)
                print(
                    f"Torrent successfully removed from session cause of 2 hours runtime: {torrent_name}"
                )
                continue

            # check if torrent still downloading
            if not torrent.get_status().is_seeding:
                continue

            else:
                torrent.move_video_files(destination)
                torrent.remove_torrent(ses)
                torrent.remove_the_link_from_the_list(magnet_links)
                update_txt_file(magnet_links_txt, magnet_links)
                torrent_name = torrent.name
                downloads.remove(torrent)
                print(f"Torrent successfully downloaded: {torrent_name}")
                gc.collect()

        print(
            f"{time.strftime('%H:%M:%S')} - Number of torrents downloading: {len(downloads)}"
        )
        time.sleep(300)

In [ ]:
# give the path of the txt file
magnet_links_txt = r"/content/drive/MyDrive/Torrents/magnet_links.txt"

# give the path of the destination folder
destination = r"/content/drive/MyDrive/Pics/Torrent"

# global downlads list
downloads = []
settings = {
    "user_agent": f"python_client/{lt.__version__}",
    "listen_interfaces": "0.0.0.0:6881",
}

In [ ]:
def main():
    start_time = time.time()

    ses = lt.session(settings)

    # load magnet links from txt file
    magnet_links = load_magnet_links(magnet_links_txt)

    # get magnet links from user input
    magnet_links = get_magnet_links(magnet_links)

    # update txt file
    update_txt_file(magnet_links_txt, magnet_links)

    # use global downloads list to keep track of torrents
    create_torrents(ses, magnet_links, downloads)

    # check if torrent already downloaded
    check_already_downladed(ses, downloads, destination, magnet_links, magnet_links_txt)

    # start downloading
    download_torrents(ses, downloads, destination, magnet_links, magnet_links_txt)

    # Calculate and display the total time taken
    end_time = time.time()
    elapsed_time_minutes = (end_time - start_time) // 60
    print(f"Torrents download completed in {elapsed_time_minutes} minutes.")